In [3]:
import pandas as pd
df=pd.read_csv('5dataSeH_model.csv')

In [4]:
df.shape

(186, 3991)

In [5]:
from sklearn.linear_model import LinearRegression
from testset_prediction import testset_prediction as tsp
from sequential_selection import stepwise_selection as sq
from loo import loo
reg=LinearRegression()

def process(tr,trX,Xts,y,yts,seed):
    lt=[]
    ls=['r2','neg_mean_absolute_error','neg_mean_poisson_deviance','neg_mean_gamma_deviance']
    #ls=['r2']
    lf=[0,5]
    #lf=[0]
    l1,l2,l3,l4,l5,l6,l7=[],[],[],[],[],[],[]
    for i in ls:
        for j in lf:
            sqs=sq(trX,y,8,True,True,i,j)
            try:
               a1,b1=sqs.fit_()
               reg.fit(tr[a1],y)
               r2tr=reg.score(tr[a1],y)
               cv=loo(tr[a1],y,tr)
               print(a1)
               c,m,l=cv.cal()
               r2pr,r2pr2,RMSEP=testpred(Xts[a1],yts,reg,m)
               rm2tr,drm2tr=rm2(y,l).fit()
               ytspr=pd.DataFrame(reg.predict(Xts[a1]))
               rm2ts,drm2ts=rm2(yts,ytspr).fit()
            except ValueError:
                c=0
                r2pr=0
                rm2tr=0
                rm2ts=0         
            #print(c,m,l)
            #print(a1)
            l1.append(i) 
            l2.append(j)  
            l3.append(c)
            l4.append(r2pr)
            l6.append(rm2tr)
            l7.append(rm2ts)
            l5.append(seed)
    Dict=dict([('random_seed', l5),('score', l1),('fold', l2),('Q2LOO', l3), ('R2Pred', l4), ('rm2tr', l6), ('rm2ts', l7)])
    print(Dict)
    table=pd.DataFrame(Dict)
    return table
    #tbname='Results_table.csv'
    #table.to_csv('Results_table.csv',index=False) 

In [6]:
from sklearn.feature_selection import VarianceThreshold
def correlation(X,cthreshold):
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = X.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (abs(corr_matrix.iloc[i, j]) > float(cthreshold)) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
                if colname in X.columns:
                   del X[colname] # deleting the column from the dataset
    return X   

def variance(X,threshold):
    sel = VarianceThreshold(threshold=(threshold* (1 - threshold)))
    sel_var=sel.fit_transform(X)
    X=X[X.columns[sel.get_support(indices=True)]]    
    return X


def pretreat(X,cthreshold,vthreshold):
    X=correlation(X,cthreshold)
    X=variance(X,vthreshold)
    return X

In [7]:
def testpred(Xts,yts,model,trav):
    ytspr=pd.DataFrame(model.predict(Xts))
    ytspr.columns=['Pred']
    tsdf=pd.concat([yts,pd.DataFrame(ytspr)],axis=1)
    tsdf.columns=['Active','Predict']
    tsdf['Aver']=trav
    tsdf['Aver2']=tsdf['Predict'].mean()
    tsdf['diff']=tsdf['Active']-tsdf['Predict']
    tsdf['diff2']=tsdf['Active']-tsdf['Aver']
    tsdf['diff3']=tsdf['Active']-tsdf['Aver2']
    r2pr=1-((tsdf['diff']**2).sum()/(tsdf['diff2']**2).sum())
    r2pr2=1-((tsdf['diff']**2).sum()/(tsdf['diff3']**2).sum())
    RMSEP=((tsdf['diff']**2).sum()/tsdf.shape[0])**0.5
    return r2pr,r2pr2,RMSEP

In [ ]:
import pandas as pd
X=df.iloc[:,2:]
Xpt=pretreat(X,0.99,0.0001)

In [15]:
from sklearn.model_selection import train_test_split
import pandas as pd
Xpt2=Xpt.columns.tolist()[1:]
perc=20
nls=[]
for sp in range(2,5):
    #perc=int(secondEntryTabOne.get())
    pc=int(100/perc)
    #sp=int(secondEntryTabOne_x.get())
    filem=df.sort_values(df.iloc[:,1:2].columns[0],ascending=False)
    ts=filem.iloc[(sp-1)::pc, :]
    tr=filem.drop(ts.index.values)
    #tr.to_csv('tr.csv', index=False)
    #a,b= train_test_split(df,test_size=perc, random_state=i)
    #tr=pd.DataFrame(a)
    #ts=pd.DataFrame(b)
    tr=tr.reset_index().drop('index', axis=1)
    ts=ts.reset_index().drop('index', axis=1)
    trX=tr[Xpt2]
    Xts=ts[Xpt2]
    #print(trX.shape)
    y=tr.iloc[:,1:2]
    tr1=pd.concat([tr.iloc[:,0:2],trX],axis=1)
    ts1=pd.concat([ts.iloc[:,0:2],Xts],axis=1)
    #tr1.to_csv('tr.csv', index=False)
    #ts1.to_csv('ts.csv', index=False)
    yts=ts.iloc[:,1:2] 
    table=process(tr,trX,Xts,y,yts,sp)
    nls.append(table)
    fd=pd.concat(nls, axis=0)

C:\Users\amitc\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['Del_Res']=dataf[self.y.columns[0]]-dataf['Pred_loo']
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['aver']=

C:\Users\amitc\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['Del_Res']=dataf[self.y.columns[0]]-dataf['Pred_loo']
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['aver']=

C:\Users\amitc\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['Del_Res']=dataf[self.y.columns[0]]-dataf['Pred_loo']
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['aver']=

C:\Users\amitc\anaconda3\lib\site-packages\statsmodels\tsa\tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['Del_Res']=dataf[self.y.columns[0]]-dataf['Pred_loo']
C:\Users\amitc\Documents\subham_work\SFS-QSAR-tool-master\loo.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  dataf['aver']=

In [16]:
fd['Average']=fd[['Q2LOO', 'R2Pred']].mean(axis=1)
fd.sort_values('Average', ascending=False)

,start_point,score,fold,Q2LOO,R2Pred,Average
4,4,neg_mean_poisson_deviance,0,0.709382,0.562686,0.636034
0,4,r2,0,0.709382,0.562686,0.636034
3,3,neg_mean_absolute_error,5,0.587652,0.527151,0.557402
0,3,r2,0,0.714010,0.317279,0.515644
7,2,neg_mean_gamma_deviance,5,0.725024,0.260477,0.492750
3,4,neg_mean_absolute_error,5,0.313697,0.651372,0.482534
7,4,neg_mean_gamma_deviance,5,0.657342,0.253554,0.455448
5,4,neg_mean_poisson_deviance,5,0.657342,0.253554,0.455448
2,4,neg_mean_absolute_error,0,0.704509,0.190100,0.447305
0,2,r2,0,0.707002,0.185993,0.446498
